In [1]:
from pyspark.sql import SparkSession,SQLContext,Row
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import StringIndexer

In [2]:
sc = SparkSession.builder.appName('ML_Model').getOrCreate() 

sq = SQLContext(sparkContext=sc.sparkContext, sparkSession=sc)

24/02/01 03:33:05 WARN Utils: Your hostname, wafa-VirtualBox resolves to a loopback address: 127.0.1.1; using 192.168.56.101 instead (on interface enp0s3)
24/02/01 03:33:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/01 03:33:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = sq.read.csv(f'hdfs://192.168.56.102:8020/user/cloudera/heart_2022_no_nans.csv', inferSchema ='true' ,header='true',sep=',')

In [4]:
'''df_pand = df.toPandas()
df_pand.shape
print("sex= ",df_pand["sex"].unique())
print("generalHealth =  " ,df_pand["generalHealth"].unique())
print("PhysicalHealthDays= ",df_pand["PhysicalHealthDays"].unique())
print("physicalActivities= ",df_pand["physicalActivities"].unique())
print("removedTeeth= ",df_pand["removedTeeth"].unique())
print("hadAngina= ",df_pand["hadAngina"].unique())
print("hadStroke= ",df_pand["hadStroke"].unique())
print("hadCOPD= ",df_pand["hadCOPD"].unique())
print("hadKidneyDisease= ",df_pand["hadKidneyDisease"].unique())
print("hadArthritis= ",df_pand["hadArthritis"].unique())
print("hadDiabetes= ",df_pand["hadDiabetes"].unique())
print("deafOrHardOfHearing= ",df_pand["deafOrHardOfHearing"].unique())
print("blindOrVisionDifficulty= ",df_pand["blindOrVisionDifficulty"].unique())
print("difficultyWalking= ",df_pand["difficultyWalking"].unique())
print("difficultyDressingBathing= ",df_pand["difficultyDressingBathing"].unique())
print("difficultyErrands= ",df_pand["difficultyErrands"].unique())
print("smokerStatus= ",df_pand["smokerStatus"].unique())
print("chestScan= ",df_pand["chestScan"].unique())
print("ageCategory= ",df_pand["ageCategory"].unique())
print("alcoholDrinkers= ",df_pand["alcoholDrinkers"].unique())
print("pneumoVaxEver= ",df_pand["pneumoVaxEver"].unique())'''

'df_pand = df.toPandas()\ndf_pand.shape\nprint("sex= ",df_pand["sex"].unique())\nprint("generalHealth =  " ,df_pand["generalHealth"].unique())\nprint("PhysicalHealthDays= ",df_pand["PhysicalHealthDays"].unique())\nprint("physicalActivities= ",df_pand["physicalActivities"].unique())\nprint("removedTeeth= ",df_pand["removedTeeth"].unique())\nprint("hadAngina= ",df_pand["hadAngina"].unique())\nprint("hadStroke= ",df_pand["hadStroke"].unique())\nprint("hadCOPD= ",df_pand["hadCOPD"].unique())\nprint("hadKidneyDisease= ",df_pand["hadKidneyDisease"].unique())\nprint("hadArthritis= ",df_pand["hadArthritis"].unique())\nprint("hadDiabetes= ",df_pand["hadDiabetes"].unique())\nprint("deafOrHardOfHearing= ",df_pand["deafOrHardOfHearing"].unique())\nprint("blindOrVisionDifficulty= ",df_pand["blindOrVisionDifficulty"].unique())\nprint("difficultyWalking= ",df_pand["difficultyWalking"].unique())\nprint("difficultyDressingBathing= ",df_pand["difficultyDressingBathing"].unique())\nprint("difficultyErran

In [5]:
df.printSchema()


root
 |-- sex: string (nullable = true)
 |-- generalHealth: string (nullable = true)
 |-- PhysicalHealthDays: double (nullable = true)
 |-- physicalActivities: string (nullable = true)
 |-- removedTeeth: string (nullable = true)
 |-- hadAngina: string (nullable = true)
 |-- hadStroke: string (nullable = true)
 |-- hadCOPD: string (nullable = true)
 |-- hadKidneyDisease: string (nullable = true)
 |-- hadArthritis: string (nullable = true)
 |-- hadDiabetes: string (nullable = true)
 |-- hadHeartAttack: string (nullable = true)
 |-- deafOrHardOfHearing: string (nullable = true)
 |-- blindOrVisionDifficulty: string (nullable = true)
 |-- difficultyWalking: string (nullable = true)
 |-- difficultyDressingBathing: string (nullable = true)
 |-- difficultyErrands: string (nullable = true)
 |-- smokerStatus: string (nullable = true)
 |-- chestScan: string (nullable = true)
 |-- ageCategory: string (nullable = true)
 |-- alcoholDrinkers: string (nullable = true)
 |-- pneumoVaxEver: string (nulla

In [6]:
input = ["sex","generalHealth","PhysicalHealthDays","physicalActivities","removedTeeth","hadAngina","hadHeartAttack","hadStroke","hadCOPD","hadKidneyDisease","hadArthritis",
                 "hadDiabetes","deafOrHardOfHearing","blindOrVisionDifficulty","difficultyWalking","difficultyDressingBathing","difficultyErrands","smokerStatus","chestScan",
                  "ageCategory","alcoholDrinkers","pneumoVaxEver"]

output = ["Sex","GeneralHealth","physicalHealthDays","PhysicalActivities","RemovedTeeth","HadAngina","HadHeartAttack","HadStroke","HadCOPD","HadKidneyDisease","HadArthritis",
                 "HadDiabetes","DeafOrHardOfHearing","BlindOrVisionDifficulty","DifficultyWalking","DifficultyDressingBathing","DifficultyErrands","SmokerStatus","ChestScan",
                  "AgeCategory","AlcoholDrinkers","PneumoVaxEver"]

encoder = StringIndexer(inputCols=input,outputCols=output)
df_encoded = encoder.fit(df).transform(df)

24/02/01 03:34:50 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [7]:
features_cls = output = ["Sex","GeneralHealth","physicalHealthDays","PhysicalActivities","RemovedTeeth","HadAngina","HadStroke","HadCOPD","HadKidneyDisease","HadArthritis",
                 "HadDiabetes","DeafOrHardOfHearing","BlindOrVisionDifficulty","DifficultyWalking","DifficultyDressingBathing","DifficultyErrands","SmokerStatus","ChestScan",
                  "AgeCategory","AlcoholDrinkers","PneumoVaxEver"]
vector_assembler = VectorAssembler(inputCols=features_cls, outputCol="features")
df_encoded = vector_assembler.transform(df_encoded)
df_encoded 


DataFrame[Sex: double, GeneralHealth: double, physicalHealthDays: double, PhysicalActivities: double, RemovedTeeth: double, HadAngina: double, HadStroke: double, HadCOPD: double, HadKidneyDisease: double, HadArthritis: double, HadDiabetes: double, HadHeartAttack: double, DeafOrHardOfHearing: double, BlindOrVisionDifficulty: double, DifficultyWalking: double, DifficultyDressingBathing: double, DifficultyErrands: double, SmokerStatus: double, ChestScan: double, AgeCategory: double, AlcoholDrinkers: double, PneumoVaxEver: double, features: vector]

In [8]:
train_data, test_data = df_encoded.randomSplit([0.8, 0.2], seed=123)

In [9]:
#Decision Tree Model
dt = DecisionTreeClassifier(labelCol="HadHeartAttack", featuresCol="features")
model = dt.fit(train_data)
predictions = model.transform(test_data)

In [10]:
evaluator = BinaryClassificationEvaluator(labelCol="HadHeartAttack", rawPredictionCol="prediction", metricName="areaUnderROC")
accuracy = evaluator.evaluate(predictions)

In [11]:
print(accuracy)

0.5913249635963841


In [12]:
# the logistic regression model

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

target_column = "HadHeartAttack"

lr = LogisticRegression(maxIter=100, regParam=0.01, labelCol=target_column)

# Fit the model 
model2 = lr.fit(train_data)
# Make predictions 
predict = model2.transform(test_data)

# Evaluate the model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol=target_column, metricName="areaUnderROC")
accuracy = evaluator.evaluate(predict)
print(f"accuracy: {accuracy}")


24/02/01 03:36:23 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


accuracy: 0.8811438299345558


In [37]:
evaluator = BinaryClassificationEvaluator(labelCol=target_column, rawPredictionCol="rawPrediction")

# Area Under ROC
roc_auc = evaluator.evaluate(predict)
print(f"accuracy: {roc_auc}")

true_positive = predict.filter((predict[target_column] == 1) & (predict['prediction'] == 1)).count()
false_positive = predict.filter((predict[target_column] == 0) & (predict['prediction'] == 1)).count()

precision = true_positive / (true_positive + false_positive)
print(f"Precision: {precision}")

Area Under ROC: 0.8811359872374415


Precision: 0.5537270087124879
